In [6]:
from itertools import combinations

import os
import sys
from pathlib import Path

sys.path.append(str((Path(os.path.abspath("")) / "..").resolve()))
from sudoku_variants import Sudoku, SudokuAI, SudokuConst
from sudoku_variants.rule import Orthogonal, SubBoard, Knight, King, Consecutive
from sudoku_variants.func import rules as R
from sudoku_variants.helper.profile import profile


def standard_rules():
    return [Orthogonal(), SubBoard()]


def variant_rules():
    return [Knight(), King(), Consecutive()]

### AI

In [13]:
@profile(10)
def run(rules):
    board = [[0 for _ in range(SudokuConst.NUM_COL)] for _ in range(SudokuConst.NUM_ROW)]
    sudoku = Sudoku(board, rules)
    SudokuAI.solve(sudoku)


for length in range(len(variant_rules()) + 1):
    for subset in combinations(variant_rules(), length):
        print(R.to_name(list(subset)))
        rules = standard_rules() + list(subset)
        run(rules)


Time: 0.0299s, 10x | 0.0030s/time
Knight
Time: 3.0598s, 10x | 0.3060s/time
King
Time: 0.1844s, 10x | 0.0184s/time
Consecutive
Time: 0.0297s, 10x | 0.0030s/time
Knight, King
Time: 0.8084s, 10x | 0.0808s/time
Knight, Consecutive
Time: 7.0052s, 10x | 0.7005s/time
King, Consecutive
Time: 0.1038s, 10x | 0.0104s/time
Knight, King, Consecutive
Time: 1.0852s, 10x | 0.1085s/time


In [24]:
@profile(10)
def run(rules, to_erase):
    SudokuAI.generate(rules, max_erased=to_erase)


for to_erase in [35, 40, 50, 60]:
    print(to_erase)
    rules = [Orthogonal(), SubBoard(), Knight(), King()]
    run(rules, to_erase)

35
Time: 0.6332s, 10x | 0.0633s/time
40
Time: 1.1341s, 10x | 0.1134s/time
50
Time: 1.3481s, 10x | 0.1348s/time
60
Time: 1.4380s, 10x | 0.1438s/time


In [39]:
@profile(1)
def run():
    sudoku = SudokuAI.generate([Orthogonal(), SubBoard(), Knight(), King(), Consecutive()], seed=1)
    print(sudoku)


run()